In [70]:
#取得日数の指定(先に選択されている想定))
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')

url = 'https://stock-price-app-miyasaka.herokuapp.com/'
browser.get(url)

In [89]:
#指定された日付を取得（addeventlistnerは組めないっぽい）
selectDays = browser.find_element_by_class_name('Select-value')
selectDays = selectDays.text
selectDays

'180(約6ヶ月分)'

In [90]:
if selectDays == '20(約20日分)':
    selectDay = 20
elif selectDays == '60(約2ヶ月分)':
    selectDay = 60
elif selectDays == '180(約6ヶ月分)':
    selectDay = 180
elif selectDays == '400(約1年分)':
    selectDay = 400
elif selectDays == '800(約2年分)':
    selectDay = 800
elif selectDays == '1200(約3年分)':
    selectDay = 1200
else:
    print('error')   
selectDay

180

In [91]:
import math
i = math.ceil(selectDay/20)-1

8

In [92]:
url2 = 'https://minkabu.jp/stock/100000018/daily_bar'
browser.get(url2)

In [93]:
#自動更新を止める
btn = browser.find_element_by_xpath('//*[@id="h_top"]/ul/li[1]')
stop = btn.click()

In [94]:
#さらに表示ボタンを押す 繰り返し処理
#クリックを押すのが早すぎて要素が見つかっていない -> time.sleep(秒)：指定時間待機
import time
for _ in range(i):
    more = browser.find_element_by_class_name('moreContent,tac,mt10')
    more.click()
    time.sleep(0.5)

In [3]:
#要素抽出
elems = browser.find_element_by_id('fourvalue_timeline')
elems = elems.find_elements_by_tag_name('tr')

In [4]:
elems[19].text

'2020/01/15 23,923.48 23,997.39 23,875.82 23,916.58 23,916.58'

In [71]:
dates = []
starts = []
highs = []
lows = []
ends = []
adjusteds = []

def extract():
    for elem in elems[1:]:      #list[start:end]
        data = elem.text.split(" ")
        
        date = data[0]
        start = data[1]
        high = data[2]
        low = data[3]
        end = data[4]
        adjusted = data[5]

        dates.append(date)
        starts.append(start)
        highs.append(high)
        lows.append(low)
        ends.append(end)
        adjusteds.append(adjusted)


In [17]:
#要素抽出
elems = browser.find_element_by_id('fourvalue_timeline')
elems = elems.find_elements_by_tag_name('tr')

In [18]:
dates = []
starts = []
highs = []
lows = []
ends = []
adjusteds = []

def extract():
    for elem in elems[1:]:      #list[start:end]
        data = elem.text.split(" ")
        
        date = data[0]
        start = data[1]
        high = data[2]
        low = data[3]
        end = data[4]
        adjusted = data[5]

        dates.append(date)
        starts.append(start)
        highs.append(high)
        lows.append(low)
        ends.append(end)
        adjusteds.append(adjusted)

In [19]:
extract()

In [22]:
#表にする
import pandas as pd
df = pd.DataFrame()

In [23]:
df['date']=dates
df['start']=starts
df['high']=highs
df['low']=lows
df['end']=ends
df['adjusted']=adjusteds

In [27]:
df.head()

,date,start,high,low,end,adjusted
0,2020/02/10,"23,631.79","23,788.25","23,621.72","23,685.98","23,685.98"
1,2020/02/07,"23,899.01","23,943.45","23,759.42","23,827.98","23,827.98"
2,2020/02/06,"23,641.10","23,995.37","23,625.13","23,873.59","23,873.59"
3,2020/02/05,"23,351.47","23,414.69","23,241.64","23,319.56","23,319.56"
4,2020/02/04,"22,881.13","23,118.13","22,854.45","23,084.59","23,084.59"


In [28]:
#inputボタンの設置（取得日付の入力）
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [33]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(
        children='Stock price scrayping app',
        style = {
            'textAlign':'center',
            'fontSize':'1.8em'
            }
        ),

    html.Div(
        children='''
        -------------------------------------------------------------
    ''',
            style = {
            'textAlign':'center'
            }
    ),
    html.Label('取得する過去データの日数',
        style = {'textAlign':'center'}
        ),

    dcc.Dropdown(
        options =[
            {'label':'20日分','value':'20'},
            {'label':'2ヶ月分','value':'60'},
            {'label':'6ヶ月分','value':'180'},
            {'label':'1年分','value':'400'},
            {'label':'2年分','value':'800'},
            {'label':'3年分','value':'1200'}
        ],
        value = '400',
        style = {'maxWidth':'40%',
                'margin':'0 auto'        
        }
    )
])

# if __name__ == '__main__':
#     app.run_server(debug=True)